In [14]:
import yaml
import requests
import base64
import json
from pymongo import MongoClient
import time
from datetime import datetime,timedelta
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [3]:
sid = SentimentIntensityAnalyzer()

In [4]:
def load_api_key(filename='twitter_api_key.yaml'):
    """Load Yelp API client ID and client secret and return them as a dictionary."""
    with open(filename) as f:
        return yaml.load(f)
    
key = load_api_key()

In [5]:
client_key = key['consumer_key']
client_secret = key['consumer_secret']

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [6]:
base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

auth_data = {
    'grant_type': 'client_credentials'
}

auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

In [7]:
auth_resp.status_code

200

In [8]:
auth_resp.json().keys()

dict_keys(['token_type', 'access_token'])

In [9]:
access_token = auth_resp.json()['access_token']

In [10]:
search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

In [11]:
search_params = {
    'q': 'kroger',
    'lang':'en',
    'count': 2,
    'tweet_mode':'extended',
    'geocode':"37.781157,-122.398720,500mi"
}

search_url = '{}1.1/search/tweets.json'.format(base_url)

search_resp = requests.get(search_url, headers=search_headers, params=search_params)

In [13]:
res = json.loads( search_resp.content )
res['statuses']

[{'created_at': 'Wed Mar 27 21:00:16 +0000 2019',
  'id': 1111010066528911360,
  'id_str': '1111010066528911360',
  'full_text': 'Walgreens, Kroger Lose Drug Pricing Suit Against J&amp;J https://t.co/Spfrbky9zx',
  'truncated': False,
  'display_text_range': [0, 80],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/Spfrbky9zx',
     'expanded_url': 'https://lnkd.in/ee5JXEv',
     'display_url': 'lnkd.in/ee5JXEv',
     'indices': [57, 80]}]},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="http://www.linkedin.com/" rel="nofollow">LinkedIn</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 148680026,
   'id_str': '148680026',
   'name': 'Edouard Lassalle',
   'screen_name': 'Elassalle',
   'location': 'San Francisco, CA',
   'description': 'Business

In [11]:
url = 'https://api.twitter.com/1.1/application/rate_limit_status.json'

search_resp = requests.get(url, headers=search_headers)

In [12]:
json.loads(search_resp.content)['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1553747857}}

In [17]:
healthy = pd.read_csv('data/healthy.csv', header = None)

In [21]:
healthy_foods = list(healthy[0])

In [25]:
client = MongoClient()
db = client['capstone']
coll=db['healthy']

In [ ]:
#words = ['safeway','albertsons','ralphs','kroger','trader joe\'s','whole foods','jewel-osco', 'pavilions', 'food 4 less','randalls','tom thumb','star market','vons','united supermarkets','acme markets', 'carrs']
words = healthy_foods

req_count = 0
for query in words:
    print("starting: ",query)
    current = datetime.now()
    Prior = datetime.now() - timedelta(days=2)
    c = 0
    lst = []
    s = []
    q = query + " -filter:retweets"
    while current>Prior:
        if c == 0:
            search_params = {
                'q': q,
                'lang':'en',
                'count': 100,
                'tweet_mode':'extended'
                #,'geocode':"41.878485,-87.627893,100mi"
            }

            search_url = '{}1.1/search/tweets.json'.format(base_url)

            search_resp = requests.get(search_url, headers=search_headers, params=search_params)
            
            Query = json.loads( search_resp.content )
            
            #Query = t.search.tweets(q=q,lang='en', count=100,tweet_mode='extended',geocode="41.878485,-87.627893,100mi")['statuses']
            #Query = t.search.tweets(q=q,result_type='recent',count=100,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
        else:
            search_params = {
                'q': q,
                'lang':'en',
                'count': 100,
                'tweet_mode':'extended',
                'max_id': max_id
                #,'geocode':"41.878485,-87.627893,100mi",
            }

            search_url = '{}1.1/search/tweets.json'.format(base_url)

            search_resp = requests.get(search_url, headers=search_headers, params=search_params)
            
            Query = json.loads( search_resp.content )
                        
            #Query = t.search.tweets(q=q,result_type='recent',count=100,max_id=max_id,tweet_mode='extended')
            next_cursor = Query['search_metadata']['next_results']
            c+=1
            
        for i,tweet in enumerate(Query['statuses']):
            if tweet['id'] not in s and query in tweet['full_text'].lower():
                date = tweet['created_at']
                current = datetime.strptime(date, '%a %b %d %H:%M:%S +0000 %Y')
                lst.append(tweet['full_text'])
                s.append(tweet['id'])
                ss = sid.polarity_scores(tweet['full_text'])
                coll.insert_one({'full_data': tweet,'text': tweet['full_text'],'keyword':query, 'sentiment':ss})
        req_count+=1
        if req_count % 50 == 0:
            print("current request count: ", req_count)
        max_id = tweet['id']
        if req_count>448:
            print("sleep time")
            time.sleep(60*15)
            req_count = 0
print("finished pull")

starting:  almond
current request count:  50
starting:  apple
current request count:  100
current request count:  150
current request count:  200
current request count:  250
current request count:  300
current request count:  350
current request count:  400
sleep time
current request count:  50
current request count:  100
current request count:  150
current request count:  200
current request count:  250
current request count:  300
current request count:  350
current request count:  400
sleep time
current request count:  50
current request count:  100
current request count:  150
current request count:  200
current request count:  250
current request count:  300
current request count:  350
current request count:  400
sleep time
